In [67]:
import SimpleITK as sitk
import numpy as np
import csv
from glob import glob
import pandas as pd
from skimage import morphology
from skimage import measure
from sklearn.cluster import KMeans
from skimage.transform import resize
from tqdm import tqdm
import skimage
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
#import pydicom
import scipy.misc


In [68]:
def get_filename(case):
  global file_list
  for f in file_list:
    if case in f:
      return(f)

In [69]:
subset="subset1"
luna_path="C:\\Users\\P70065719\\Desktop\\Thesis\\dataset\\LUNA\\1\\"
output_path="C:\\Users\\P70065719\\Desktop\\Thesis\\output\\"
file_list=glob(luna_path+"data\\"+subset+"\\"+"*.mhd")
annotations_path=luna_path+"annotations.csv"

In [70]:
len(file_list)

89

In [71]:
df_node=pd.read_csv(annotations_path)
df_node["file"]=df_node["seriesuid"].apply(get_filename)
df_node=df_node.dropna()


In [72]:
len(df_node)

128

In [73]:
def make_mask(center,diam,z,width,height,spacing,origin):
    '''
Center : centers of circles px -- list of coordinates x,y,z
diam : diameters of circles px -- diameter
widthXheight : pixel dim of image
spacing = mm/px conversion rate np array x,y,z
origin = x,y,z mm np.array
z = z position of slice in world coordinates mm
    '''
    mask = np.zeros([height,width]) # 0's everywhere except nodule swapping x,y to match img
    #convert to nodule space from world coordinates

    # Defining the voxel range in which the nodule falls
    v_center = (center-origin)/spacing
    v_diam = int(diam/spacing[0]+5)
    v_xmin = np.max([0,int(v_center[0]-v_diam)-5])
    v_xmax = np.min([width-1,int(v_center[0]+v_diam)+5])
    v_ymin = np.max([0,int(v_center[1]-v_diam)-5]) 
    v_ymax = np.min([height-1,int(v_center[1]+v_diam)+5])

    v_xrange = range(v_xmin,v_xmax+1)
    v_yrange = range(v_ymin,v_ymax+1)

    # Convert back to world coordinates for distance calculation
    x_data = [x*spacing[0]+origin[0] for x in range(width)]
    y_data = [x*spacing[1]+origin[1] for x in range(height)]

    # Fill in 1 within sphere around nodule
    for v_x in v_xrange:
        for v_y in v_yrange:
            p_x = spacing[0]*v_x + origin[0]
            p_y = spacing[1]*v_y + origin[1]
            if np.linalg.norm(center-np.array([p_x,p_y,z]))<=diam:
                mask[int((p_y-origin[1])/spacing[1]),int((p_x-origin[0])/spacing[0])] = 1.0
    return(mask)

def matrix2int16(matrix):
    ''' 
matrix must be a numpy array NXN
Returns uint16 version
    '''
    m_min= np.min(matrix)
    m_max= np.max(matrix)
    matrix = matrix-m_min
    return(np.array(np.rint( (matrix-m_min)/float(m_max-m_min) * 65535.0),dtype=np.uint16))


In [74]:
# def normalizePlanes(npzarray):
#         maxHU = 400.
#         minHU = -1000.
#         npzarray = (npzarray - minHU) / (maxHU - minHU)
#         npzarray[npzarray>1] = 1.
#         npzarray[npzarray<0] = 0.
#         return npzarray

In [75]:
img_list={}
import math
prefix=luna_path+"data\\"+subset+"\\"
f=0

In [76]:
for img_file in file_list:
    #img_file='drive/My Drive/Thesis/dataset/subset0/1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524522225658609808059.mhd'
    mini_df = df_node[df_node["file"]==img_file] 
    #get all nodules associate with file
    if mini_df.shape[0]>0: # some files may not have a nodule--skipping those 
        # load the data once
        print(f)
        f=f+1
        itk_img = sitk.ReadImage(img_file) 
        img_array = sitk.GetArrayFromImage(itk_img) # indexes are z,y,x (notice the ordering)
        num_z, height, width = img_array.shape        #heightXwidth constitute the transverse plane
        origin = np.array(itk_img.GetOrigin())      # x,y,z  Origin in world coordinates (mm)
        spacing = np.array(itk_img.GetSpacing())
        #print(spacing)# spacing of voxels in world coor. (mm)
        # go through all nodes (why just the biggest?)
#         maxHU = 400.
#         minHU = -1000.
#         img_array = (img_array - minHU) / (maxHU - minHU)
#         img_array[img_array>1] = 1.
#         img_array[img_array<0] = 0.
        t_num=-1 #Tumor number of patient
        for node_idx, cur_row in mini_df.iterrows():       
            node_x = cur_row["coordX"]
            node_y = cur_row["coordY"]
            node_z = cur_row["coordZ"]
            diam = cur_row["diameter_mm"]
            #print(diam)
            # just keep 3 slices
            t_num=t_num+1
            positive_slices=math.ceil(diam/spacing[2])
            imgs = np.ndarray([5,height,width],dtype=np.float32)
            masks = np.ndarray([5,height,width],dtype=np.uint8)
            center = np.array([node_x, node_y, node_z])   # nodule center
            v_center = np.rint((center-origin)/spacing)  # nodule center in voxel space (still x,y,z ordering)
            
            
#             for i, i_z in enumerate(np.arange(int(v_center[2])-int(positive_slices),
#                              int(v_center[2])-int(positive_slices/2)).clip(0, num_z-1)):
#                 fname=img_file[len(prefix):-4]+"_"+str(i_z)+"_"+str(0)
#                 img_list[fname] = img_array[i_z]
                
                
            for i, i_z in enumerate(np.arange(int(v_center[2])-2,
                             int(v_center[2])+3).clip(0, num_z-1)): # clip prevents going out of bounds in Z
                flag=0
                if i_z in range(int(v_center[2])-int(positive_slices/2),int(v_center[2])+int(positive_slices/2)):
                    flag=1
                fname=img_file[len(prefix):-4]+"_"+str(t_num)+"_"+str(i_z)+"_"+str(flag)
                img_list[fname] = img_array[i_z]
    if img_list:
        break
            
#             for i, i_z in enumerate(np.arange(int(v_center[2])+int(positive_slices/2),
#                              int(v_center[2])+int(positive_slices)).clip(0, num_z-1)):
#                 fname=img_file[len(prefix):-4]+"_"+str(i_z)+"_"+str(0)
#                 img_list[fname] = img_array[i_z]
  
#             np.save(os.path.join(output_path,"images_%04d_%04d.npy" % (fcount, node_idx)),imgs)
#             np.save(os.path.join(output_path,"masks_%04d_%04d.npy" % (fcount, node_idx)),masks)

0


In [77]:
img_list

{'1.3.6.1.4.1.14519.5.2.1.6279.6001.104562737760173137525888934217_0_102_1': array([[-1000, -1000, -1000, ..., -1000, -1000, -1000],
        [-1000, -1000, -1000, ..., -1000, -1000, -1000],
        [-1000, -1000, -1000, ..., -1000, -1000, -1000],
        ...,
        [-1000, -1000, -1000, ..., -1000, -1000, -1000],
        [-1000, -1000, -1000, ..., -1000, -1000, -1000],
        [-1000, -1000, -1000, ..., -1000, -1000, -1000]], dtype=int16),
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.104562737760173137525888934217_0_103_1': array([[-1000, -1000, -1000, ..., -1000, -1000, -1000],
        [-1000, -1000, -1000, ..., -1000, -1000, -1000],
        [-1000, -1000, -1000, ..., -1000, -1000, -1000],
        ...,
        [-1000, -1000, -1000, ..., -1000, -1000, -1000],
        [-1000, -1000, -1000, ..., -1000, -1000, -1000],
        [-1000, -1000, -1000, ..., -1000, -1000, -1000]], dtype=int16),
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.104562737760173137525888934217_0_104_1': array([[-1000, -1000, -1000, ..

In [78]:

# import matplotlib.pyplot as plt
# imgs[imgs<=-1000]=0
# for i in range(len(imgs)):
#     print ("image %d" % i)
#     fig,ax = plt.subplots(2,2,figsize=[8,8])
#     ax[0,0].imshow(imgs[i],cmap=plt.cm.bone)
#     ax[0,1].imshow(masks[i],cmap=plt.cm.bone)
#     ax[1,0].imshow(imgs[i]*masks[i],cmap=plt.cm.bone)
#     plt.show()
#     input("hit enter to cont : ")

In [79]:




prefix=luna_path+"data\\"+subset+"\\"
print(len(img_list))
# img_list=[]

5


In [80]:
# for img_file in file_list:
#   itk_img=sitk.ReadImage(img_file)
#   img_array=sitk.GetArrayFromImage(itk_img)
#   img_array[img_array<=-2000]=0
#   img_list.append(img_array)

In [81]:
# len(img_list)

In [82]:
lung_mask={}

In [83]:
import warnings
warnings.simplefilter('error')
f=0

In [84]:
for j in img_list :
    # I ran into an error when using Kmean on np.float16, so I'm using np.float64 here
#     imgs_to_process = img_list[j].astype(np.float64)
#     print( f)
#     f=f+1
#     for i in range(len(imgs_to_process)):

        imgs_to_process = img_list[j].astype(np.float64)
        #Standardize the pixel values
        img=imgs_to_process
        mean = np.mean(img)
        std = np.std(img)
        img = img-mean
        img = img/std if std!=0 else img
        # Find the average pixel value near the lungs
        # to renormalize washed out images
        middle = img[100:400,100:400] 
        mean = np.mean(middle)  
        max = np.max(img)
        min = np.min(img)
        # To improve threshold finding, I'm moving the 
        # underflow and overflow on the pixel spectrum
        img[img==max]=mean
        img[img==min]=mean
        #
        # Using Kmeans to separate foreground (radio-opaque tissue)
        # and background (radio transparent tissue ie lungs)
        # Doing this only on the center of the image to avoid 
        # the non-tissue parts of the image as much as possible
        #
        #warnings.simplefilter('error')
        try:
          
          kmeans = KMeans(n_clusters=2).fit(np.reshape(middle,[np.prod(middle.shape),1]))
          centers = sorted(kmeans.cluster_centers_.flatten())
          threshold = np.mean(centers)
          thresh_img = np.where(img<threshold,1.0,0.0)  # threshold the image
        except:
          imgs_to_process=np.zeros(shape=img.shape)
          continue
        #warnings.simplefilter('ignore')
        #
        # I found an initial erosion helful for removing graininess from some of the regions
        # and then large dialation is used to make the lung region 
        # engulf the vessels and incursions into the lung cavity by 
        # radio opaque tissue
        #
        eroded = morphology.erosion(thresh_img,np.ones([4,4]))
        dilation = morphology.dilation(eroded,np.ones([10,10]))
        #
        #  Label each region and obtain the region properties
        #  The background region is removed by removing regions 
        #  with a bbox that is to large in either dimnsion
        #  Also, the lungs are generally far away from the top 
        #  and bottom of the image, so any regions that are too
        #  close to the top and bottom are removed
        #  This does not produce a perfect segmentation of the lungs
        #  from the image, but it is surprisingly good considering its
        #  simplicity. 
        #
        labels = measure.label(dilation)
        label_vals = np.unique(labels)
        regions = measure.regionprops(labels)
        good_labels = []
        for prop in regions:
            B = prop.bbox
            if B[2]-B[0]<475 and B[3]-B[1]<475 and B[0]>40 and B[2]<472:
                good_labels.append(prop.label)
        mask = np.ndarray([512,512],dtype=np.int8)
        mask[:] = 0
        #
        #  The mask here is the mask for the lungs--not the nodes
        #  After just the lungs are left, we do another large dilation
        #  in order to fill in and out the lung mask 
        #
        for N in good_labels:
            mask = mask + np.where(labels==N,1,0)
        mask = morphology.dilation(mask,np.ones([10,10])) # one last dilation
        imgs_to_process = mask
        lung_mask[j]=imgs_to_process
   

In [85]:
f=0
import os

In [86]:
for i in lung_mask:
    print(f)
    f=f+1
#     file_path=i.split('_')
#     fname=output_path+file_path[0]+"\\"+file_path[1]+"\\"
#     if not os.path.exists(fname):
#         os.makedirs(fname)
# #     fname=output_path+i
    img=img_list[i]
    lung=lung_mask[i]
#     img_file=fname+file_path[2]+"_"+file_path[3]+".png"
    final_img=img*lung
    maxHU = 400.
    minHU = -1000.
    final_img = (final_img - minHU) / (maxHU - minHU)
    final_img[final_img>1] = 1.
    final_img[final_img<0] = 0.
    img=(img-minHU)/(maxHU-minHU)
    img[img>1]=1
    img[img<0]=0
    plt.imsave('img1.png',img,cmap=plt.cm.gray)
    plt.imsave('mask1.png',lung,cmap=plt.cm.gray)
    plt.imsave('seg1.png',final_img,cmap=plt.cm.gray)
    
    
#     plt.imsave(img_file,final_img,cmap=plt.cm.gray)

0
1
2
3
4


In [66]:
len(lung_mask)

5